# Fase 07 — Deploy & Runtime Validation (Notebook)
 
Este notebook ejecuta la Fase 07:

- Arranca un servidor Flask de inferencia.
- Envía **solo ventanas de observación únicas** (optimización fuerte).
- Recoge predicciones de todos los modelos del paquete F06.
- Calcula métricas por modelo (TP, TN, FP, FN, precision, recall, f1).
- Genera un informe HTML simple y figuras de matriz de confusión.


In [1]:
import os

PHASE = "07_deployrun"
VARIANT = os.environ.get("ACTIVE_VARIANT", "v701")

import sys
import os
from pathlib import Path
import json
from datetime import datetime, timezone
from time import perf_counter
import shutil
import yaml
import pyarrow.parquet as pq

SCRIPT_PATH = Path.cwd().resolve()
ROOT = SCRIPT_PATH
for _ in range(10):
    if (ROOT / "mlops4ofp").exists():
        break
    ROOT = ROOT.parent
else:
    raise RuntimeError("No se pudo localizar project root")

sys.path.insert(0, str(ROOT))
print("Project root:", ROOT)

Project root: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp


In [2]:
import time
import traceback

import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

from flask import Flask, request, jsonify
from werkzeug.serving import make_server
import tensorflow as tf
import yaml

from mlops4ofp.tools.params_manager import ParamsManager
from mlops4ofp.tools.run_context import detect_execution_dir, detect_project_root
from mlops4ofp.tools.traceability import write_metadata


In [3]:
PHASE = "07_deployrun"

# Variante activa: leída del entorno (make nb7-run establece ACTIVE_VARIANT)
VARIANT = os.environ.get("ACTIVE_VARIANT", None)
if not VARIANT:
    raise RuntimeError("ACTIVE_VARIANT no definido. Ejecuta con make nb7-run VARIANT=v7XX")

print(f"[CTX] Fase: {PHASE}  Variante: {VARIANT}")

# Detectar contexto de ejecución
execution_dir = detect_execution_dir()
project_root = detect_project_root(execution_dir)

print(f"[CTX] execution_dir = {execution_dir}")
print(f"[CTX] project_root   = {project_root}")

pm = ParamsManager(PHASE, project_root)
pm.set_current(VARIANT)
variant_root = pm.current_variant_dir()

print(f"[CTX] variant_root   = {variant_root}")

# Cargar params.yaml
params_path = variant_root / "params.yaml"
if not params_path.exists():
    raise FileNotFoundError(f"No existe params.yaml en {params_path}")

with open(params_path, "r") as f:
    params = yaml.safe_load(f)

parent_f06 = params["parent_variant_f06"]
batch_size = params.get("batch_size", 256)
sample_size = params.get("sample_size", None)
infer_batch_retries = params.get("infer_batch_retries", 3)
progress_every_batches = params.get("progress_every_batches", 10)

print(f"[CFG] parent_f06             = {parent_f06}")
print(f"[CFG] batch_size             = {batch_size}")
print(f"[CFG] sample_size            = {sample_size}")
print(f"[CFG] infer_batch_retries    = {infer_batch_retries}")
print(f"[CFG] progress_every_batches = {progress_every_batches}")

# Directorios de artefactos
runtime_dir = variant_root / "runtime"
logs_dir = variant_root / "logs"
metrics_dir = variant_root / "metrics"
report_dir = variant_root / "report"
figures_dir = report_dir / "figures"

for d in [runtime_dir, logs_dir, metrics_dir, report_dir, figures_dir]:
    d.mkdir(parents=True, exist_ok=True)

# Manifest
manifest_path = variant_root / "manifest.json"
if not manifest_path.exists():
    raise FileNotFoundError(f"manifest.json no encontrado en {manifest_path}. Ejecuta make variant7 antes.")

with open(manifest_path, "r") as f:
    manifest = json.load(f)

print(f"[MANIFEST] F06: {manifest.get('f06_variant', parent_f06)}")
print(f"[MANIFEST] modelos empaquetados: {len(manifest['models'])}")
print(f"[MANIFEST] datasets F04: {len(manifest['datasets'])}")


[CTX] Fase: 07_deployrun  Variante: v701
[CTX] execution_dir = /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/notebooks
[CTX] project_root   = /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp
[CTX] variant_root   = /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701
[CFG] parent_f06             = v601
[CFG] batch_size             = 512
[CFG] sample_size            = None
[CFG] infer_batch_retries    = 3
[CFG] progress_every_batches = 10
[MANIFEST] F06: v601
[MANIFEST] modelos empaquetados: 4
[MANIFEST] datasets F04: 4


In [4]:
def to_json_safe_window(window):
    """Normaliza la ventana OW_events a lista nativa para poder serializarla."""
    if hasattr(window, "tolist"):
        return window.tolist()
    if isinstance(window, (list, tuple)):
        return list(window)
    if hasattr(window, "item"):
        return [window.item()]
    return [window]

# Paths para logs crudos
raw_path_parquet = logs_dir / "raw_predictions.parquet"
raw_path_csv = logs_dir / "raw_predictions.csv"

def write_batch_predictions(batch_df: pd.DataFrame):
    """Append incremental de logs crudos en CSV y Parquet.

    Dado que el número de ventanas únicas es relativamente pequeño,
    podemos permitirnos read+concat+write para Parquet.
    """
    # CSV (append)
    if not raw_path_csv.exists():
        batch_df.to_csv(raw_path_csv, index=False)
    else:
        batch_df.to_csv(raw_path_csv, index=False, mode="a", header=False)

    # Parquet (read+concat+overwrite; coste asumible por tamaño reducido)
    if not raw_path_parquet.exists():
        batch_df.to_parquet(raw_path_parquet, index=False)
    else:
        existing = pd.read_parquet(raw_path_parquet)
        combined = pd.concat([existing, batch_df], ignore_index=True)
        combined.to_parquet(raw_path_parquet, index=False)


In [5]:
HOST = "127.0.0.1"
PORT = 5005
base_url = f"http://{HOST}:{PORT}"

def build_flask_app(manifest: dict) -> Flask:
    app = Flask(__name__)

    loaded_models = []

    for m in manifest["models"]:
        model_dir = Path(m["model_dir"])
        summary_path = model_dir / m["model_summary"]
        model_path = model_dir / m["model_h5"]

        summary = json.loads(summary_path.read_text())
        model = tf.keras.models.load_model(model_path)

        loaded_models.append({
            "prediction_name": summary["prediction_name"],
            "model": model,
            "vectorization": summary["vectorization"],
            "threshold": summary.get("threshold", 0.5),
        })

    def vectorize_batch(windows, cfg):
        if cfg["vectorization"] == "dense_bow":
            vocab = cfg["vocab"]
            index = {ev: i for i, ev in enumerate(vocab)}
            X = np.zeros((len(windows), cfg["input_dim"]), dtype=np.float32)
            for i, window in enumerate(windows):
                for ev in window:
                    if ev in index:
                        X[i, index[ev]] += 1.0
            return X

        if cfg["vectorization"] == "sequence":
            vocab = cfg["vocab"]
            index = {ev: i + 1 for i, ev in enumerate(vocab)}
            max_len = cfg["max_len"]
            X = np.zeros((len(windows), max_len), dtype=np.int32)
            for i, window in enumerate(windows):
                seq = [index[e] for e in window if e in index]
                seq = seq[-max_len:]
                if len(seq) > 0:
                    X[i, -len(seq):] = seq
            return X

        raise ValueError("vectorization no soportada: " + str(cfg.get("vectorization")))

    @app.route("/", methods=["GET"])
    def health():
        return jsonify({"status": "ready", "models": len(loaded_models)})

    @app.route("/infer_batch", methods=["POST"])
    def infer_batch():
        payload = request.get_json(force=True)
        windows = payload["windows"]

        batch_results = []

        for m in loaded_models:
            X_batch = vectorize_batch(windows, m["vectorization"])
            y_probs = m["model"].predict(X_batch, verbose=0).flatten()
            y_preds = (y_probs >= m["threshold"]).astype(int)

            for i, window in enumerate(windows):
                if len(batch_results) <= i:
                    batch_results.append({"window": window, "results": []})

                batch_results[i]["results"].append({
                    "prediction_name": m["prediction_name"],
                    "y_pred": int(y_preds[i]),
                })

        return jsonify({"results": batch_results})

    @app.route("/control", methods=["POST"])
    def control():
        payload = request.get_json(force=True) or {}
        if payload.get("cmd") == "shutdown":
            func = request.environ.get("werkzeug.server.shutdown")
            if func:
                func()
            return jsonify({"status": "shutting_down"})
        return jsonify({"status": "unknown_command"})

    @app.errorhandler(Exception)
    def handle_exception(err):
        tb = traceback.format_exc()
        return jsonify({"error": str(err), "traceback": tb}), 500

    return app

class ServerThread:
    def __init__(self, app: Flask, host: str, port: int):
        self.host = host
        self.port = port
        self._server = make_server(host, port, app)
        self._ctx = app.app_context()
        self._ctx.push()

    def start(self):
        import threading
        self._thread = threading.Thread(target=self._server.serve_forever, daemon=True)
        self._thread.start()

    def shutdown(self):
        self._server.shutdown()
        if hasattr(self, "_thread"):
            self._thread.join(timeout=2.0)

# Arrancar servidor
app = build_flask_app(manifest)
server_thread = ServerThread(app, HOST, PORT)
server_thread.start()

# Esperar a que responda
for attempt in range(10):
    try:
        r = requests.get(base_url + "/", timeout=0.5)
        if r.status_code == 200:
            print("[SERVER] Flask listo:", r.json())
            break
    except Exception:
        time.sleep(0.5)
else:
    raise RuntimeError("Servidor Flask no respondió al healthcheck.")


127.0.0.1 - - [19/Feb/2026 18:26:54] "GET / HTTP/1.1" 200 -


[SERVER] Flask listo: {'models': 4, 'status': 'ready'}


In [6]:
def post_infer_batch_with_retry(windows, dataset_name, processed_count, total_count):
    last_error = None

    for attempt in range(1, infer_batch_retries + 1):
        try:
            resp = requests.post(
                f"{base_url}/infer_batch",
                json={"windows": windows},
                timeout=120,
            )
            resp.raise_for_status()
            if attempt > 1:
                print(
                    f"[INFO] infer_batch recuperado tras {attempt} intentos "
                    f"en {dataset_name} ({processed_count}/{total_count})",
                    flush=True,
                )
            return resp.json()
        except requests.RequestException as err:
            last_error = err
            print(
                f"[WARN] infer_batch fallo intento {attempt}/{infer_batch_retries} "
                f"en {dataset_name} ({processed_count}/{total_count}): {err}",
                flush=True,
            )

            if attempt == infer_batch_retries:
                raise RuntimeError(
                    "infer_batch agotó reintentos. "
                    f"Dataset={dataset_name}, progreso={processed_count}/{total_count}, "
                    f"último_error={last_error}"
                ) from err

            backoff_seconds = min(2 ** (attempt - 1), 5)
            time.sleep(backoff_seconds)


In [7]:
raw_df = None

try:
    # =============================
    # 1) Dataset base (único)
    # =============================
    base_dataset = manifest["datasets"][0]
    df = pd.read_parquet(base_dataset["dataset_path"])

    if sample_size:
        df = df.head(sample_size)

    x_column = base_dataset["x_column"]
    dataset_name = Path(base_dataset["dataset_path"]).name

    print(f"\n[INFO] Dataset base: {dataset_name}", flush=True)
    print(f"[INFO] Filas totales: {len(df)}", flush=True)

    # =============================
    # 2) Serializar y deduplicar ventanas
    # =============================
    print("[INFO] Serializando ventanas...", flush=True)

    df["window"] = df[x_column].apply(
        lambda w: json.dumps(
            to_json_safe_window(w),
            separators=(",", ":"),
            ensure_ascii=False,
        )
    )

    unique_windows = df["window"].unique()
    total_unique = len(unique_windows)

    print(f"[INFO] Ventanas únicas detectadas: {total_unique}", flush=True)
    print(f"[INFO] Reducción: {len(df)} → {total_unique}", flush=True)
    print(f"[INFO] batch_size={batch_size}", flush=True)

    # =============================
    # 3) Inferencia sobre ventanas únicas
    # =============================
    processed_unique = 0
    progress_stride = max(batch_size * progress_every_batches, 1)

    print("\n[INFO] Iniciando inferencia sobre ventanas únicas...\n", flush=True)

    # Limpiar logs previos si existían
    if raw_path_parquet.exists():
        raw_path_parquet.unlink()
    if raw_path_csv.exists():
        raw_path_csv.unlink()

    for i in range(0, total_unique, batch_size):
        batch_window_json = unique_windows[i:i + batch_size]

        batch_windows = [json.loads(w) for w in batch_window_json]

        data = post_infer_batch_with_retry(
            windows=batch_windows,
            dataset_name=dataset_name,
            processed_count=min(i + batch_size, total_unique),
            total_count=total_unique,
        )

        batch_rows = []
        for item in data["results"]:
            window_json = json.dumps(
                item["window"], separators=(",", ":"), ensure_ascii=False
            )
            for r in item["results"]:
                batch_rows.append({
                    "window": window_json,
                    "prediction_name": r["prediction_name"],
                    "y_pred": r["y_pred"],
                })

        batch_df = pd.DataFrame(batch_rows)
        write_batch_predictions(batch_df)

        processed_unique += len(batch_windows)

        if processed_unique % progress_stride == 0 or processed_unique == total_unique:
            pct = processed_unique / total_unique * 100
            print(
                f"[RUN] Ventanas únicas procesadas "
                f"{processed_unique}/{total_unique} ({pct:.1f}%)",
                flush=True,
            )

    print("\n[INFO] Inferencia completada.", flush=True)

    # =============================
    # 4) Cargar logs crudos
    # =============================
    if raw_path_parquet.exists():
        raw_df = pd.read_parquet(raw_path_parquet)
    else:
        raw_df = pd.DataFrame(columns=["window", "prediction_name", "y_pred"])

    print("\n[OK] logs crudos guardados:", flush=True)
    print(" -", raw_path_parquet, flush=True)
    print(" -", raw_path_csv, flush=True)
    print("[OK] filas únicas inferidas:", len(raw_df), flush=True)

finally:
    # Intentar apagar servidor limpiamente
    try:
        requests.post(f"{base_url}/control", json={"cmd": "shutdown"}, timeout=5)
    except Exception:
        pass
    try:
        server_thread.shutdown()
    except Exception:
        pass



[INFO] Dataset base: v401__dataset.parquet


[INFO] Filas totales: 2895405


[INFO] Serializando ventanas...


[INFO] Ventanas únicas detectadas: 310521


[INFO] Reducción: 2895405 → 310521


...


[INFO] Inferencia completada.



[OK] logs crudos guardados:


 - /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/logs/raw_predictions.parquet


 - /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/logs/raw_predictions.csv


[OK] filas únicas inferidas: 1242084


In [8]:
if raw_df is None or raw_df.empty:
    print("[WARN] raw_df vacío; no se pueden calcular métricas.")
else:
    print("\n[INFO] Calculando métricas por modelo...", flush=True)

    metrics = []

    for m in manifest["models"]:
        pred_name = m["prediction_name"]
        dataset_path = m["dataset_path"]

        df_model = pd.read_parquet(dataset_path)
        if sample_size:
            df_model = df_model.head(sample_size)

        # Serializar ventana para merge
        df_model["window"] = df_model["OW_events"].apply(
            lambda w: json.dumps(
                to_json_safe_window(w),
                separators=(",", ":"),
                ensure_ascii=False,
            )
        )

        model_log = raw_df[raw_df["prediction_name"] == pred_name][["window", "y_pred"]]

        merged = df_model.merge(model_log, on="window", how="left")

        valid = merged[merged["y_pred"].notna()].copy()
        if valid.empty:
            print(f"[WARN] Sin predicciones válidas para {pred_name}, se omite.", flush=True)
            continue

        valid["y_pred"] = valid["y_pred"].astype(int)

        tp = ((valid["label"] == 1) & (valid["y_pred"] == 1)).sum()
        tn = ((valid["label"] == 0) & (valid["y_pred"] == 0)).sum()
        fp = ((valid["label"] == 0) & (valid["y_pred"] == 1)).sum()
        fn = ((valid["label"] == 1) & (valid["y_pred"] == 0)).sum()

        precision = tp / (tp + fp) if (tp + fp) else 0.0
        recall = tp / (tp + fn) if (tp + fn) else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0

        metrics.append({
            "prediction_name": pred_name,
            "tp": int(tp),
            "tn": int(tn),
            "fp": int(fp),
            "fn": int(fn),
            "precision": float(precision),
            "recall": float(recall),
            "f1": float(f1),
        })

        # Figura de matriz de confusión
        plt.figure()
        plt.imshow([[tn, fp], [fn, tp]])
        plt.title(f"Confusion Matrix - {pred_name}")
        plt.colorbar()
        plt.xticks([0, 1], ["Pred 0", "Pred 1"])
        plt.yticks([0, 1], ["True 0", "True 1"])
        plt.tight_layout()
        fig_path = figures_dir / f"confusion_{pred_name}.png"
        plt.savefig(fig_path)
        plt.close()
        print(f"[FIG] Guardada matriz de confusión: {fig_path}", flush=True)

    if metrics:
        metrics_df = pd.DataFrame(metrics)
        metrics_csv_path = metrics_dir / "metrics_per_model.csv"
        metrics_df.to_csv(metrics_csv_path, index=False)
        print(f"[OK] Métricas guardadas en {metrics_csv_path}", flush=True)

        # Informe HTML simple
        report_html = "<html><body><h1>F07 Report</h1>"
        report_html += "<h2>Métricas por modelo</h2>"
        report_html += metrics_df.to_html(index=False)
        report_html += "</body></html>"

        report_path = report_dir / "report.html"
        report_path.write_text(report_html, encoding="utf-8")
        print(f"[OK] Reporte HTML: {report_path}", flush=True)
    else:
        print("[WARN] No se generaron métricas.", flush=True)



[INFO] Calculando métricas por modelo...


[FIG] Guardada matriz de confusión: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/report/figures/confusion_battery_active_power_any-to-80_100.png


[FIG] Guardada matriz de confusión: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/report/figures/confusion_battery_active_power_set_response_any-to-80_100.png


[FIG] Guardada matriz de confusión: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/report/figures/confusion_pvpcs_active_power_any-to-80_100.png


[FIG] Guardada matriz de confusión: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/report/figures/confusion_ge_active_power_any-to-80_100.png


[OK] Métricas guardadas en /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/metrics/metrics_per_model.csv


[OK] Reporte HTML: /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/report/report.html


In [9]:
# Registrar trazabilidad de la fase
metadata_path = variant_root / f"{PHASE}_metadata.json"

write_metadata(
    stage=PHASE,
    variant=VARIANT,
    parent_variant=manifest.get("f06_variant", parent_f06),
    inputs=[str(manifest_path)],
    outputs=[str(raw_path_parquet), str(metrics_dir), str(report_dir)],
    params=params,
    metadata_path=metadata_path,
)

print(f"[TRACE] Metadata de F07 escrita en {metadata_path}")


[TRACE] Metadata de F07 escrita en /Users/juancarlosduenaslopez/Documents/mlops/mlops4ofp/executions/07_deployrun/v701/07_deployrun_metadata.json
